# Fit model parameters

In [2]:
import random

import numpy as np
import pandas as pd
import seaborn as sns

from construal_shifting.task_modeling.model_fitter import ModelFitter
from construal_shifting.task_modeling.participant_model import ParticipantModel

from data_analysis import download_data, download_condition_counts, calc_condition_counts, ExperimentDataLoader


In [3]:
exp_data = ExperimentDataLoader(
    trialdata_file="rawtrialdata-anon.csv"
)
summary_df = pd.read_json('./data/summary_df.json')
all_participant_data = [p for p in exp_data.completed_participant_data() if p.sessionId in summary_df.sessionId.values]

Participant participant_021:anonymized missing trials
Trial Indices = [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]; expected [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]
Participant participant_096:anonymized missing trials
Trial Indices = [0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]; expected [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


In [4]:
fixed_params=dict(
    construal_cost_weight=.0,
    construal_set_stickiness=.0,
    construal_inverse_temp=1.0,
    action_inverse_temp=float('inf'),
    action_random_choice=.1,
)
param_bounds = dict(
    construal_set_stickiness=(0., 10.),
    construal_cost_weight=(0., 10.),
    construal_inverse_temp=(0., 10),
    action_inverse_temp=(1e-2, 10),
    action_random_choice=(.05, 1.), #we need to lower bound to avoid numerical issues
)
param_combos_to_fit = [
    ["action_random_choice"],
    ["construal_cost_weight", "construal_set_stickiness", "action_random_choice"],
    ["construal_set_stickiness", "action_random_choice"],
    ["construal_cost_weight", "action_random_choice"],
]

results = []
rng = random.Random(61311)
fitter = ModelFitter(all_participant_data)
for params_to_fit in param_combos_to_fit:
    fit_res = fitter.fit_params(
        fixed_params={p: v for p, v in fixed_params.items() if p not in params_to_fit},
        params_to_fit=params_to_fit,
        param_bounds=param_bounds,
        maxfun=200,
        runs=3,
        seed=rng.randint(0, int(1e7))
    )
    for i, res in enumerate(fit_res):
        results.append({
            "run": i,
            **res.model_params,
            **{'fit_'+p: True for p in params_to_fit},
            'NLL': res.neg_log_like,
        })

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [8]:
import pandas as pd
res_df = pd.DataFrame(results)
res_df['df'] = res_df[['fit_construal_cost_weight', 'fit_construal_set_stickiness', 'fit_action_random_choice']].apply(lambda r: sum([t == True for t in r]), axis=1)
res_df['AIC'] = res_df.apply(lambda r: 2*r['df'] + r['NLL'], axis=1)
res_df['dAIC'] = res_df['AIC'] - res_df['AIC'].min()
optimal_results = res_df.loc[[0, 9, 6, 3]].reset_index(drop=True)
optimal_results

,run,construal_cost_weight,construal_set_stickiness,construal_inverse_temp,action_inverse_temp,action_random_choice,fit_action_random_choice,NLL,fit_construal_cost_weight,fit_construal_set_stickiness,df,AIC,dAIC
0,0,0.000000,0.0,1.0,inf,0.096484,True,95929.323918,NaN,NaN,1,95931.323918,239.432474
1,0,0.078727,0.0,1.0,inf,0.096485,True,95926.342563,True,NaN,2,95930.342563,238.451119
2,0,0.000000,10.0,1.0,inf,0.096215,True,95758.436889,NaN,True,2,95762.436889,70.545445
3,0,0.642017,10.0,1.0,inf,0.096034,True,95685.891443,True,True,3,95691.891443,0.000000


In [9]:
optimal_results.to_pickle('./data/model_fits.pkl')